In [1]:
from pointnet2.utils.pointnet2_modules import PointnetSAModuleMSG, PointnetSAModule

In [2]:
from torch.autograd import Variable
import torch

In [3]:
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [4]:
xyz = Variable(torch.randn(2, 9, 3).cuda(), requires_grad=True)
xyz_feats = Variable(torch.randn(2, 9, 6).cuda(), requires_grad=True)

In [5]:
print(xyz)

tensor([[[-1.5256, -0.7502, -0.6540],
         [-1.6095, -0.1002, -0.6092],
         [-0.9798, -1.6091, -0.7121],
         [ 0.3037, -0.7773, -0.2515],
         [-0.2223,  1.6871,  0.2284],
         [ 0.4676, -0.6970, -1.1608],
         [ 0.6995,  0.1991,  0.8657],
         [ 0.2444, -0.6629,  0.8073],
         [ 1.1017, -0.1759, -2.2456]],

        [[-1.4465,  0.0612, -0.6177],
         [-0.7981, -0.1316,  1.8793],
         [-0.0721,  0.1578, -0.7735],
         [ 0.1991,  0.0457,  1.1651],
         [ 2.0154,  0.2152, -0.5242],
         [-1.8034, -1.3083,  0.4533],
         [ 1.1422, -3.3312, -0.7479],
         [ 1.1173,  0.2981,  0.1099],
         [-0.6463,  0.4285,  1.4761]]], device='cuda:0', requires_grad=True)


In [6]:
test_module = PointnetSAModuleMSG(
    npoint=2, radii=[5.0, 10.0], nsamples=[6, 3], mlps=[[9, 3], [9, 6]]
)
test_module.cuda()
print(test_module(xyz, xyz_feats))

#  test_module = PointnetFPModule(mlp=[6, 6])
#  test_module.cuda()
#  from torch.autograd import gradcheck
#  inputs = (xyz, xyz, None, xyz_feats)
#  test = gradcheck(test_module, inputs, eps=1e-6, atol=1e-4)
#  print(test)

for _ in range(1):
    _, new_features = test_module(xyz, xyz_feats)
    new_features.backward(torch.cuda.FloatTensor(*new_features.size()).fill_(1))
    print(new_features)
#     print(xyz.grad)

(tensor([[[-1.5256, -0.7502, -0.6540],
         [ 1.1017, -0.1759, -2.2456]],

        [[-1.4465,  0.0612, -0.6177],
         [ 1.1422, -3.3312, -0.7479]]], device='cuda:0',
       grad_fn=<CloneBackward>), tensor([[[0.1114, 0.9845],
         [0.8835, 1.8192],
         [1.1286, 0.9887],
         [0.0000, 0.6805],
         [1.8098, 0.9786],
         [0.9279, 1.1666],
         [0.4875, 0.7851],
         [1.0264, 0.7702],
         [0.8286, 0.0000]],

        [[2.1496, 2.3403],
         [1.1343, 1.9554],
         [1.8369, 0.7857],
         [0.0000, 1.6078],
         [0.6667, 0.8107],
         [1.0543, 0.0524],
         [1.1522, 0.0000],
         [1.2703, 0.4379],
         [1.2366, 1.6281]]], device='cuda:0', grad_fn=<CatBackward>))
tensor([[[0.1114, 0.9845],
         [0.8835, 1.8192],
         [1.1286, 0.9887],
         [0.0000, 0.6805],
         [1.8098, 0.9786],
         [0.9279, 1.1666],
         [0.4875, 0.7851],
         [1.0264, 0.7702],
         [0.8286, 0.0000]],

        [[2.1496,

# Try to get 128 features

## 1. Load checkpionts

##  2. Setup model

## 3. Output features

In [7]:
from pointnet2.models.pointnet2_msg_cls import Pointnet2MSG
from pointnet2.models.pointnet2_msg_cls import model_fn_decorator
import torch.nn as nn
import torchvision

%load_ext autoreload
%autoreload 2

In [8]:
model = Pointnet2MSG(input_channels=0, num_classes=40, use_xyz=True)

In [9]:
model.cuda()

Pointnet2MSG(
  (SA_modules): ModuleList(
    (0): PointnetSAModuleMSG(
      (groupers): ModuleList(
        (0): QueryAndGroup()
        (1): QueryAndGroup()
        (2): QueryAndGroup()
      )
      (mlps): ModuleList(
        (0): SharedMLP(
          (layer0): Conv2d(
            (conv): Conv2d(3, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer1): Conv2d(
            (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (normlayer): BatchNorm2d(
              (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            )
            (activation): ReLU(inplace)
          )
          (layer2): Conv2d(
            (conv): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (nor

In [10]:
model_fn = model_fn_decorator(nn.CrossEntropyLoss())

In [11]:
# use MdoelNet40Cls to see what are the input points format
import pointnet2.data.data_utils as d_utils
transforms = torchvision.transforms.Compose(
        [
            d_utils.PointcloudToTensor(),
            d_utils.PointcloudScale(),
            d_utils.PointcloudRotate(),
            d_utils.PointcloudRotatePerturbation(),
            d_utils.PointcloudTranslate(),
            d_utils.PointcloudJitter(),
            d_utils.PointcloudRandomInputDropout(),
        ]
    )

from pointnet2.data import ModelNet40Cls
num_points = 50000
train_set = ModelNet40Cls(num_points, transforms=transforms)


from torch.utils.data import DataLoader
batch_size = 1
train_loader = DataLoader(
        train_set,
        batch_size=batch_size,
        shuffle=True,
        num_workers=2,
        pin_memory=True,
)

In [12]:
train_set[0][0].numpy()

array([[-0.25385302, -0.32723886,  0.06503981],
       [-0.25385302, -0.32723886,  0.06503981],
       [-0.25385302, -0.32723886,  0.06503981],
       ...,
       [-0.25385302, -0.32723886,  0.06503981],
       [-0.24375083,  0.07920968, -0.06177698],
       [-0.25385302, -0.32723886,  0.06503981]], dtype=float32)

In [13]:
from matplotlib import pyplot
from mpl_toolkits.mplot3d import Axes3D
import random

fig = pyplot.figure()
ax = Axes3D(fig)

data_to_draw = train_set[0][0].numpy()

sequence_containing_x_vals = data_to_draw[:, 0]
sequence_containing_y_vals = data_to_draw[:, 1]
sequence_containing_z_vals = data_to_draw[:, 2]

ax.scatter(sequence_containing_x_vals, sequence_containing_y_vals, sequence_containing_z_vals)
pyplot.show()

<Figure size 640x480 with 1 Axes>

In [14]:
# create a point(cloud) for above evaluations code

In [15]:
# eval the model and print features
model.eval()
for data in train_loader:
    _, loss, eval_res = model_fn(model, data, eval=True)
    break

In [16]:
f1 = model.get_features()
f1.unsqueeze(-1)
print(f1.cpu().numpy()[0, 0:100])

[[0.26790357]
 [0.        ]
 [0.        ]
 [0.37726364]
 [0.33487338]
 [0.8299709 ]
 [0.4946904 ]
 [0.030349  ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.        ]
 [0.11286031]
 [0.45479786]
 [0.04346748]
 [0.119688  ]
 [1.0395243 ]
 [0.70429385]
 [0.27868018]
 [0.45446604]
 [1.0125841 ]
 [0.08449543]
 [0.        ]
 [0.23594567]
 [0.        ]
 [0.15520291]
 [0.07605541]
 [0.43721718]
 [0.67015326]
 [0.        ]
 [0.23632905]
 [0.5777415 ]
 [0.3134924 ]
 [0.13102804]
 [0.58553445]
 [0.39738616]
 [0.07141984]
 [0.15555139]
 [0.        ]
 [0.20241901]
 [0.        ]
 [0.        ]
 [0.76397187]
 [0.        ]
 [0.        ]
 [0.27054942]
 [0.6010867 ]
 [0.3848288 ]
 [0.        ]
 [0.        ]
 [0.24528846]
 [0.30968735]
 [0.44619113]
 [0.6934825 ]
 [0.33882785]
 [0.02716545]
 [0.15735954]
 [0.        ]
 [0.60243064]
 [0.        ]
 [0.26516035]
 [0.06951035]
 [0.        ]
 [0.        ]
 [0.13923325]
 [0.1425806 ]
 [0.05742893]
 [0.  

In [17]:
import numpy as np
def compute_vec_dist(f1, f2):
    return np.sqrt(np.sum(np.power(f1 - f2, 2.0)))

In [18]:
dist = compute_vec_dist(f1.cpu().numpy(), f1.cpu().numpy())
print(dist)

0.0
